In [31]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer,AutoModelForSeq2SeqLM,TrainingArguments,Trainer,DataCollatorForLanguageModeling,TextDataset
from peft import LoraConfig,TaskType,get_peft_model
from datasets import load_dataset
import json

In [32]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r = 32,
    lora_alpha=32,
    lora_dropout=0.1
)
model_id = 'openai-community/gpt2'
training_args = TrainingArguments (
    output_dir='output/test',
    learning_rate=1e-3,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy='no',
    save_strategy='epoch',
    #load_best_model_at_end='True',
)

In [33]:
model = AutoModelForCausalLM.from_pretrained(model_id)
model = get_peft_model(model,peft_config)
model.print_trainable_parameters()
tokenizer = AutoTokenizer.from_pretrained(model_id)
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)


c:\Users\Anh\AppData\Local\Programs\Python\Python310\lib\site-packages\peft\tuners\lora\layer.py:861: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 1,179,648 || all params: 125,619,456 || trainable%: 0.939064725769868


In [34]:
def preprocess(input_path = 'intents.json'):
    with open(input_path,'r') as file:
        data = json.load(file)
    preprocess_data = []
    for intent in data['intents']:
        for patternn in intent['patterns']:
            preprocess_data.append(f'User: {patternn}\n')
            for response in intent['responses']:
                preprocess_data.append(f'Assistant: {response}\n')
    return ''.join(preprocess_data)
def save_preprocess(data,output_path = 'data/data.txt'):
    with open(output_path,'w') as file:
        file.write(data)

In [35]:
data = preprocess()
save_preprocess(data)

In [36]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='data/data.txt',
    block_size=64
)

c:\Users\Anh\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [25]:
print(train_dataset[64])

tensor([  318,   262,  1738,  2157,   428,    30,   198, 12982,    25,   314,
         1254,   523, 28063,    13,   198, 48902,    25,   632,   338,   691,
         3288,   284,  1254,   428,   835,    13, 14026,   502,   517,    13,
         1867,  2073,   318,   319,   534,  2000,    30,   198, 48902,    25,
         3914,   338,  2112,  2252,  1521,   345,   821,  4203,   428,   835,
           13,   198, 48902,    25,   314,   717,   765,   284,  1309,   345,
          760,   326,   345,   389])


In [38]:
trainer = Trainer(
    model = model,
    args=  training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=collator
)

In [39]:
trainer.train()

c:\Users\Anh\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/8 [00:00<?, ?it/s]

{'train_runtime': 11.236, 'train_samples_per_second': 22.695, 'train_steps_per_second': 0.712, 'train_loss': 3.0490000247955322, 'epoch': 1.0}


TrainOutput(global_step=8, training_loss=3.0490000247955322, metrics={'train_runtime': 11.236, 'train_samples_per_second': 22.695, 'train_steps_per_second': 0.712, 'train_loss': 3.0490000247955322, 'epoch': 1.0})

In [40]:
model.save_pretrained("output_dir")

In [30]:
a_model = AutoModelForCausalLM.from_pretrained("output_dir")

OSError: output_dir does not appear to have a file named config.json. Checkout 'https://huggingface.co/output_dir/None' for available files.